In [1]:
try:
  import unzip_requirements
except ImportError:
  pass

import pandas as pd
import distance
import json
import os
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import psycopg2
from geopy.geocoders import Nominatim
import boto3
from geopy.distance import geodesic
import analytics
import tweepy
from tweepy import OAuthHandler

In [2]:
analytics.write_key = os.environ.get('SEGMENT_KEY')

In [3]:
etype = []
etext = []
edate = []
jid = []

In [4]:
try:
    with SSHTunnelForwarder(
        (os.environ.get('JOUST_IP'), 22),
        ssh_private_key="~/.ssh/id_rsa",
        ssh_username="ubuntu",
        remote_bind_address=(os.environ.get('JOUST_ADDRESS'), 5432)) as server:

        server.start()
        print("server connected")

        params = {
            "database": "joust_production",
            "user": "sid",
            "password": os.environ.get('JOUST_PASS'),
            "host": "localhost",
            "port": server.local_bind_port
        }

        conn = psycopg2.connect(**params)
        curs = conn.cursor()
        print("database connected")

        plaid = "select account_holders.id, first_name,last_name, addresses.line_1,addresses.line_2,addresses.city,addresses.state,addresses.zip,mobile_phone,email,plaid_identities.updated_at,plaid_identities.raw_response->>'owners', businesses.\"name\" from account_holders right join external_bank_accounts on account_holders.id = external_bank_accounts.account_holder_id right join plaid_identities on external_bank_accounts.id = plaid_identities.plaid_identifiable_id inner join addresses on account_holders.id = addresses.addressable_id inner join businesses on account_holders.id = businesses.account_holder_id and plaid_identifiable_type='ExternalBankAccount'"
        invoices = "SELECT account_holders.id,payment_requests.payment_type,payment_requests.transaction_code,payment_requests.status,first_name,last_name,account_holders.created_at,payment_requests.paid_date,payment_requests.amount,payment_requests.created_at as created_at2, payment_requests.business_entity_payer_id from account_holders inner join payment_requests on account_holders.id = payment_requests.account_holder_id"
        status = "SELECT core_pro_customers.account_holder_id as id, promo_codes.\"name\" as promo_codes from core_pro_customers left join account_holder_promo_codes on core_pro_customers.account_holder_id = account_holder_promo_codes.account_holder_id left join promo_codes on promo_codes.id = account_holder_promo_codes.promo_code_id"
        devices = "select account_holders.id, first_name, last_name, devices.signature, devices.platform, devices.created_at from account_holders inner join account_holder_devices on account_holders.id = account_holder_devices.account_holder_id inner join devices on account_holder_devices.device_id = devices.id"
        trpay1 = "select account_holders.id as userid, payment_requests.id, account_holders.first_name,account_holders.last_name,account_holders.email,account_holders.mobile_phone from payment_requests inner join account_holders on payment_requests.account_holder_id = account_holders.id inner join addresses on payment_requests.account_holder_id = addresses.addressable_id where addressable_type = 'AccountHolder'"
        trpay2 = "select payment_requests.id, business_entity_payers.first_name, business_entity_payers.last_name, business_entity_payers.mobile_number, business_entity_payers.email, business_entities.\"name\", business_entities.phone_number, payment_requests.created_at from payment_requests inner join business_entity_payers on payment_requests.business_entity_payer_id = business_entity_payers.id left join business_entities on business_entity_payers.business_entity_id = business_entities.id inner join addresses on payment_requests.business_entity_payer_id = addresses.addressable_id where addressable_type = 'BusinessEntity'"



        plaid = pd.read_sql_query(plaid, conn)
        req = pd.read_sql_query(invoices, conn)
        phones = pd.read_sql_query(devices, conn)
        stat = pd.read_sql_query(status, conn)
        trpay1 = pd.read_sql_query(trpay1, conn)
        trpay2 = pd.read_sql_query(trpay2, conn)

        conn.close()

except Exception as e:
    print(e)
    print("Connection Failed")

server connected
database connected


In [5]:
trpay = trpay1.merge(trpay2, left_on='id', right_on='id')
trpay = trpay.fillna('')

In [6]:
plaid = plaid.rename(columns={"first_name": "First Name", "last_name": "Last Name", "?column?": "Raw Response"})
req = req.rename(columns={"first_name": "First Name", "last_name": "Last Name", "status": "Status", "amount": "Amount (Payment Requests)", "created_at": "Created At", "created_at2": "Created At (Payment Requests)", "paid_date": "Accepted Date"})
stat = stat.rename(columns={"first_name": "First Name", "last_name": "Last Name"})
phones = phones.rename(columns={"Name": "Full Name"})

In [7]:
plaid['updated_at'] =  pd.to_datetime(plaid['updated_at'])
req['Created At'] =  pd.to_datetime(req['Created At'])
req['Created At (Payment Requests)'] =  pd.to_datetime(req['Created At (Payment Requests)'])
req['Accepted Date'] =  pd.to_datetime(req['Accepted Date'])
phones['created_at'] =  pd.to_datetime(phones['created_at'])
trpay['created_at'] =  pd.to_datetime(trpay['created_at'])

In [8]:
req['Full Name'] = req['First Name'].astype(str).values + ' ' + req['Last Name'].astype(str).values
req = req.drop(['First Name','Last Name'], axis=1)
phones['Full Name'] = phones['first_name'].astype(str).values + ' ' + phones['last_name'].astype(str).values
phones = phones.drop(['first_name','last_name'], axis=1)
trpay['Full Name'] = trpay['first_name_y'].astype(str).values + ' ' + trpay['last_name_y'].astype(str).values
trpay = trpay.drop(['first_name_y','last_name_y'], axis=1)

In [9]:
plaid['now'] = datetime.utcnow()
req['now'] = datetime.utcnow()
phones['now'] = datetime.utcnow()
trpay['now'] = datetime.utcnow()

In [10]:
plaid['timediff'] = (plaid['now'] - plaid['updated_at']).dt.total_seconds() / 3600
req['timediff'] = (req['now'] - req['Created At (Payment Requests)']).dt.total_seconds() / 3600
phones['timediff'] = (phones['now'] - phones['created_at']).dt.total_seconds() / 3600
trpay['timediff'] = (trpay['now'] - trpay['created_at']).dt.total_seconds() / 3600

In [11]:
plaid = plaid.fillna('')
plaid['location'] = plaid["line_1"].astype(str) + plaid["line_2"].astype(str) + ' ' + plaid["city"].astype(str) + ', ' + plaid["zip"].astype(str)

In [12]:
newplaid = plaid[plaid['timediff'] < 10000000]

In [13]:
fids = []
ffullname = []
fnamecheck = []
lnamecheck = []
fullnamecheck = []
biznamecheck = []
nnamedist = []
biznamedist = []
minnamedist = []
statematch = []
zipmatch = []
emailmatch = []
phonematch = []
nameoncards = []
plaiddate = []
for npp in newplaid.values:
    pldate = npp[-6]
    plaiddate.append(pldate)
    plid = npp[0]
    plfname = npp[1].lower()
    pllname = npp[2].lower()
    ffullname.append(npp[1] + ' ' + npp[2])
    plstate = npp[6].lower()
    plzip = npp[7]
    plphone = npp[8]
    plemail = npp[9].lower()
    plcomp = npp[12].lower()
    pllocation = npp[-1]
    card = json.loads(npp[11])[0]
    cname = ''
    if (len(card['names']) > 0):
        cname = card['names'][0].lower()
    nameoncards.append(cname)
    cemail = ''
    if (len(card['emails']) > 0):
        cemail = card['emails'][0]['data'].lower()
    cstate = ''
    czip = ''
    clocation = ''
    if (len(card['addresses']) > 0):
        caddr = card['addresses'][0]['data']
        cstate = caddr['region'].lower()
        czip = caddr['postal_code']
        clocation = caddr['street'] + ' ' + caddr['city'] + ', ' + caddr['region'] + ' ' + caddr['postal_code']
    cphone = ''
    if (len(card['phone_numbers']) > 0):
        cphone = card['phone_numbers'][0]['data']
    fids.append(plid)
    fnamecheck.append(plfname in cname)
    lnamecheck.append(pllname in cname)
    biznamecheck.append((plcomp in cname) | (cname in plcomp))
    fullnamecheck.append((plfname in cname) & (pllname in cname))
    nnamedist.append(distance.levenshtein(plfname + ' ' + pllname,cname))
    biznamedist.append(distance.levenshtein(plcomp,cname))
    minnamedist.append(min(distance.levenshtein(plcomp,cname), distance.levenshtein(plfname + ' ' + pllname,cname)))
    statematch.append(plstate in cstate)
    zipmatch.append(plzip in czip)
    emailmatch.append(plemail in cemail)
    phonematch.append(plphone in cphone)

plaidex = pd.DataFrame(fids, columns=['id'])
plaidex['fnamecheck'] = fnamecheck
plaidex['lnamecheck'] = lnamecheck
plaidex['biznamecheck'] = biznamecheck
plaidex['nnamedist'] = nnamedist
plaidex['biznamedist'] = biznamedist
plaidex['minnamedist'] = minnamedist
plaidex['statematch'] = statematch
plaidex['zipmatch'] = zipmatch
plaidex['emailmatch'] = emailmatch
plaidex['phonematch'] = phonematch
plaidex = plaidex*1
plaidex['fullname'] = ffullname
plaidex['cardname'] = nameoncards
plaidex['date'] = plaiddate


In [14]:
for pk in plaidex.values:
    if (((pk[1] == 0) | (pk[2] == 0)) & (pk[4] >= 3)):
        pmssg = (pk[-3] + ' falsely linked their account to ' + pk[-2])
        pid = (pk[0])
        pdate = (pk[-1])
        
        etype.append('false linked account')
        etext.append(pmssg)
        edate.append(pdate)
        jid.append(pid)
        #analytics.identify(segment, {
        #    'joust_id': pid,
        #    'message': pmssg,
        #    'type': 'false linked account',
        #    'created_at': pdate
        #})

In [15]:
# add segment ids (ids copy -> ids segid replace from new table)

In [16]:
req['timediff2'] = (req['now'] - req['Accepted Date']).dt.total_seconds() / 3600
req = req.sort_values('timediff2', ascending=True)
stat = stat.groupby('id')['promo_codes'].apply(list).to_frame()
stat = stat[['promo_codes']]
req2 = req.merge(stat, left_on='id', right_on='id')
req2 = req2.sort_values('timediff2', ascending=True)

In [17]:
req3 = req2[req2['timediff2'] < 100]
for k in req3.values:
    ptype = k[1]
    if (ptype < 1):
        ptnam = 'ProPay'
    else:
        ptnam = 'ACH'
    idnum = k[0]
    pval = k[6]
    fnam = k[9]
    promo = k[13]
    if promo is None:
        paymssg = (fnam + ' (id: ' + str(idnum) + ') just got their invoice of $' + str(pval) + ' paid via ' + ptnam )
        #print(paymssg)
    else:
        paymssg = (str(fnam) + ' (id: ' + str(idnum) + ') with promo code ' + str(promo) + ' just got their invoice of $' + str(pval) + ' paid via ' + str(ptnam) )
        #print(paymssg)

In [18]:
phones2 = phones[phones['timediff'] < 100]
sigs = phones2['signature'].values
undev = []
for sig in sigs:
    undev.append(len(phones[phones['signature'] == sig]['Full Name'].unique()))
phones2['unique'] = undev
phones2 = phones2[phones2['unique'] > 1]

In [19]:
sigs = phones2['signature'].unique()
ucounts = []
unames = []
uids = []
for sig in sigs:
    phones3 = phones2[phones2['signature'] == sig]
    ucounts.append(len(phones3))
    unames.append(phones3['Full Name'].unique())
    uids.append(phones3['id'].unique())
phonefin = pd.DataFrame(sigs, columns=['signature'])
phonefin['count'] = ucounts
phonefin['names'] = unames
phonefin['ids'] = uids

In [20]:
mphone2 = ''
for k in range(len(sigs)):
    mphone2 = mphone2 + 'Device ' + sigs[k] + ' is used by ' + str(unames[k]) + ' (id: ' + str(uids[k]) + '). \n'
mphone1 = 'There were ' + str(len(phonefin)) + ' accounts with devices that have been seen before.'
if (len(phonefin) > 0):
    print(mphone1)
    print(mphone2)

In [21]:
req4 = req.sort_values('timediff', ascending=True)
req4 = req4[req4['timediff'] < 24]
dtids = req4[req4['timediff'] < 1]['id'].unique()
for num in dtids:
    req5 = req4[req4['id'] == num]
    if (len(req5) > len(req5['business_entity_payer_id'].unique())):
        print(req5.values[0])
        invm = req5.values[0][9] + ' ( id:'+ str(req5.values[0][0]) +' )' +' has sent multiple invoices to the same person in the last 24 hrs.'
        print(invm)

In [22]:
trpay2 = trpay[trpay['timediff'] < 2400000]
tpids = []
tpfullnam = []
fmat = []
lmat = []
mailmat = []
phonemat = []
tpdates = []
for tp in trpay2.values:
    tpdate = tp[-4]
    tpdates.append(tpdate)
    tpid = tp[0]
    tpids.append(tpid)
    tpfnam = tp[2].lower()
    tplnam = tp[3].lower()
    tpfullnam.append(tp[2] + ' ' + tp[3])
    tpemail = tp[4].lower()
    tpphone = tp[5]
    cpemail = tp[7].lower()
    cpphone = tp[6]
    cpfnam = tp[-3].lower()
    fmat.append(tpfnam in cpfnam)
    lmat.append(tplnam in cpfnam)
    mailmat.append(tpemail in cpemail)
    phonemat.append(tpphone in cpphone)
selfinv = pd.DataFrame(tpids, columns=['id'])
selfinv['name'] = tpfullnam
selfinv['fmat'] = fmat
selfinv['lmat'] = lmat
selfinv['mailmat'] = mailmat
selfinv['phonemat'] = phonemat
selfinv = selfinv*1
selfinv['date'] = tpdates
selfinv['fraud'] = selfinv['fmat'] + selfinv['lmat'] + selfinv['mailmat'] + selfinv['phonemat']

In [23]:
for si in selfinv[selfinv['fraud'] > 0].values:
    dupes = 'same '
    if (si[2] > 0):
        dupes = dupes + 'first name, '
    if (si[3] > 0):
        dupes = dupes + 'last name, '
    if (si[4] > 0):
        dupes = dupes + 'email, '
    if (si[5] > 0):
        dupes = dupes + 'phone, '
    smssg = (si[1] + ' (id:' + str(si[0]) + ') has sent an invoice to themself. (' + dupes[0:-2] + ')')
    sid = (si[0])
    sdate = (si[-2])
    etype.append('self invoice')
    etext.append(smssg)
    edate.append(sdate)
    jid.append(sid)
    #analytics.identify(segment, {
    #    'joust_id': sid,
    #    'message': smssg,
    #    'type': 'self invoice',
    #    'created_at': sdate
    #})

In [24]:
final = pd.DataFrame(jid, columns=['Joust ID'])
final['Event Type'] = etype
final['Event Message'] = etext
final['Event Date'] = edate